# pySudoku

This notebook contains a set of functions implemented to solve a Sudoku matrix, following the rules of the game. The algorithm implements a recursive approach to solve complex games, where multiple cells have more than one possible value.

This is an initial version of this algorithm, this means it can be improved in many ways, as no optimization has yet be performed upon the scripts.

**v1.0**


*   Base set of auxiliary functions.
*   Initial approach of a solve function, which implements an iterative approach when there are cells with only one possible value, and recursive approach when all cells have multiple possible values.
*   Test with three sudoku matrices of different difficulties.
*   Test with an empty matrix - Proves solve function can be used for creating brand new matrices.



## Global imports

In [1]:
import random
import copy

## Sudoku solving code

### Auxiliary functions


**`get_columns(sudoku)`** - Receives a sudoku and returns an array with each of the columns it has. Technically speaking, basically trasposes the sudoku matrix of shape 9x9. 

**`get_sub_matices(sudoku)`** - Receives a sudoku matrix, and returns a matrix of shape 9x9, where each main element contains the values of each sub-matrix.

**`print_sudoku(sudoku)`** - Receives a 9x9 matrix representing a sudoku, and prints it in a fancy way.

**`get_empty_sudoku()`** - Returns a 9x9 matrix representing a Sudoku, with all it's elements set as None.

In [2]:
def get_columns(sudoku):
  columns = [[],[],[],[],[],[],[],[],[]]
  for row in sudoku:
    columns[0] = columns[0] + [row[0]]
    columns[1] = columns[1] + [row[1]]
    columns[2] = columns[2] + [row[2]]
    columns[3] = columns[3] + [row[3]]
    columns[4] = columns[4] + [row[4]]
    columns[5] = columns[5] + [row[5]]
    columns[6] = columns[6] + [row[6]]
    columns[7] = columns[7] + [row[7]]
    columns[8] = columns[8] + [row[8]]
  return columns

def get_sub_matices(sudoku):
  sub_matrices = []
  for i in [0,3,6]:
    for j in [0,3,6]:
      sub_matrices.append([sudoku[i][j],
                         sudoku[i][j+1],
                         sudoku[i][j+2],
                         sudoku[i+1][j],
                         sudoku[i+1][j+1],
                         sudoku[i+1][j+2],
                         sudoku[i+2][j],
                         sudoku[i+2][j+1],
                         sudoku[i+2][j+2]])
  return sub_matrices

def print_sudoku(sudoku):
  rows_string = []
  for row in range(9):
    row_string = "│"
    for column in range(9):
      if column == 3 or column == 6:
        row_string += "│"
      if sudoku[row][column] is None:
        row_string += ' * '
      else:
        row_string += ' {} '.format(sudoku[row][column])
    row_string += '│'
    rows_string.append(row_string)
 
  sudoku_string = """
  ┌─────────┬─────────┬─────────┐
  {}
  {}
  {}
  ├─────────┼─────────┼─────────┤
  {}
  {}
  {}
  ├─────────┼─────────┼─────────┤
  {}
  {}
  {}
  └─────────┴─────────┴─────────┘
  """.format(rows_string[0],rows_string[1],rows_string[2],
             rows_string[3],rows_string[4],rows_string[5],
             rows_string[6],rows_string[7],rows_string[8])
  print(sudoku_string)

def get_empty_sudoku():
  empty_sudoku = [[None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None],
              [None,None,None,None,None,None,None,None,None]]
  return empty_sudoku

**`validate_repeated_values(array)`** - Receives an array and returns `True` or `False` 
  if a value inside the array is repeated

In [3]:
def validate_repeated_values(array):
  '''Receives an array and returns True or False 
  if a value inside the array is repeated'''
  repeated = False
  elements = {}
  for element in array:
    if element is not None:
      if element in elements.keys():
        repeated = True
        break
      else:
        elements[element] = None
  return repeated

**`is_valid_sudoku(sudoku)`** - Returns `True` if a sudoku matrix is valid. Otherwise, returns `False`. A valid sudoku will be any sudoku where values are not repeated within the same row, column or sub-matrix, and there are no cells where no value can be placed.

In [4]:
def is_valid_sudoku(sudoku):
  '''Returns if a sudoku is valid or not. Wether a sudoku is valid or not
  is defined on the repetition of values in a row, column or sub matrix. 
  Repeated values = not valid (false)
  No repeated values = valid (true)
  '''
  for row in sudoku:
    repeated_values = (validate_repeated_values(row))
    if repeated_values:
      return False
  columns = get_columns(sudoku)
  for column in columns:
    repeated_values = (validate_repeated_values(column))
    if repeated_values:
      return False
  return True
  sub_matrices = get_sub_matrices(sudoku)
  for sub_matrix in sub_matrices:
    repeated_values = (validate_repeated_values(sub_matrix))
    if repeated_values:
      return False
  return True

**`get_empty_cells(sudoku)`** - Receives a 9x9 matrix representing a Sudoku and returns an object containing all the empty cells (cells in the Sudoku matrix with `None` value) and it's possible values. The object returned will have the following form:

```
{
  'single_values'      : [],
  'multiple_values'    : [],
  'no_possible_values' : True|False
}
```
The key `'single_values'` references an array with containing all empty cells where only one value can be placed.
The key `'multiple_values'` references an array with containing all empty cells where multiple values can be placed.
The key `'no_possible_values'` is a Boolean variable which is set to `True` when a cell has no possible values (this means the Sudoku is incorrect). If all cells can have either one or multiple values, `'no_possible_values'` will be `False`.

Each empty cell in the array of single or multiple values, will be formed by a tuple with three elements: `(row_index, col_index, [POSSIBLE_VALUES])`

In [5]:
def get_empty_cells(sudoku):
  sub_matrices = get_sub_matices(sudoku)
  columns      = get_columns(sudoku)
  empty_cells  = {'single_value'       : [],
                  'multiple_values'    : [],
                  'no_possible_values' : False}

  for row_index in range(len(sudoku)):
    row = sudoku[row_index]
    for col_index in range(len(row)):
      if sudoku[row_index][col_index] == None:
        possible_values = get_possible_values(row_index,
                                              col_index,
                                              sudoku,
                                              columns,
                                              sub_matrices)
        if 'single_value' in possible_values.keys():
          empty_cells['single_value'].append((row_index,
                                                col_index,
                                                possible_values['single_value']
          ))
        elif 'no_possible_values' in possible_values.keys():
          empty_cells['no_possible_values'] = True
        else:
          empty_cells['multiple_values'].append((row_index,
                                                col_index,
                                                possible_values['multiple_values']
          ))
  empty_cells['multiple_values'].sort(key=lambda x: len(x[2]))
  return empty_cells

def get_possible_values(row_index, col_index, rows, columns, sub_matrices):
  possible_values = [1,2,3,4,5,6,7,8,9]
  sub_matrix_index = int(row_index/3) * 3 + int(col_index/3)
  for i in range(9):
    row_value        = rows[row_index][i]
    col_value        = columns[col_index][i]
    sub_matrix_value = sub_matrices[sub_matrix_index][i]
    if row_value is not None and row_value in possible_values:
      possible_values.remove(row_value)
    if col_value is not None and col_value in possible_values:
      possible_values.remove(col_value)
    if sub_matrix_value is not None and sub_matrix_value in possible_values:
      possible_values.remove(sub_matrix_value)
  if len(possible_values) == 0:
    return {'no_possible_values' : True}
  elif len(possible_values) == 1:
    return {'single_value' : possible_values}
  else:
    random.shuffle(possible_values)
    return {'multiple_values' : possible_values}

In [6]:
def fill_single_value_cells(sudoku, empty_cells):
  for empty_cell in empty_cells:
    row   = empty_cell[0]
    col   = empty_cell[1]
    value = empty_cell[2][0]
    sudoku[row][col] = value

### Main solve functions

In [7]:
def solve_sudoku(unsolved_sudoku, recursion_level = 0): 
  sudoku = copy.deepcopy(unsolved_sudoku)
  if not is_valid_sudoku(sudoku):
    return {'status': 'fail'}
  else:
    empty_cells = get_empty_cells(sudoku)
    if empty_cells['no_possible_values'] :
      return {'status': 'fail'}
    while len(empty_cells['single_value']) > 0:
      fill_single_value_cells(sudoku, empty_cells['single_value'])
      empty_cells = get_empty_cells(sudoku)
  ### BELOW THIS LINE, THERE IS TERROR ###
  still_unsolved = False
  if len(empty_cells['multiple_values'])>0:  
    #for empty_cell in empty_cells['multiple_values']:
      empty_cell_to_fill = empty_cells['multiple_values'][0]
      row_to_fill        = empty_cell_to_fill[0]
      col_to_fill        = empty_cell_to_fill[1]
      values_to_try      = empty_cell_to_fill[2]
      for value in values_to_try:
        sudoku_copy = copy.deepcopy(sudoku)
        sudoku_copy[row_to_fill][col_to_fill] = value
        result = solve_sudoku(sudoku_copy, recursion_level=recursion_level+1)
        if result['status'] is not 'fail':
          return result
  else:  
    ### ABOVE THIS LINE, THERE IS TERROR ###
    if not is_valid_sudoku(sudoku):
      return {'status': 'fail'}
    else:
      return{'status'          : 'OK',
             'sudoku'          : sudoku,
             'recursion_level' : recursion_level}
  return {'status': 'fail'}



## Tests

### Test solve function

#### Easy sudoku

In [ ]:
# sudoku = get_empty_sudoku()
sudoku_easy = [
          [None, None, None, None, 6   , 5   , 2   , None, 3   ],
          [None, None, None, 3   , 4   , 9   , 8   , None, None],
          [None, 5   , None, None, None, 8   , None, None, None],
          [3   , 7   , None, None, 2   , 4   , None, None, None],
          [None, 6   , None, None, None, 3   , 7   , None, None],
          [5   , None, None, None, 1   , None, None, None, 8   ],
          [None, 3   , None, 5   , None, None, 6   , 7   , 2   ],
          [None, None, None, None, 3   , 2   , 9   , None, 4   ],
          [7   , None, None, 4   , 8   , None, 5   , None, None]
] # EASY ONE

print('Solving an easy one...')
solved_easy = solve_sudoku(sudoku_easy)
if solved_easy['status']=='OK':
  solved_sudoku = solved_easy['sudoku']
  print(f"It is valid! Recursion depth required: {solved_easy['recursion_level']}" if is_valid_sudoku(solved_sudoku) else 'It is not valid...')
  print_sudoku(solved_sudoku)
else:
  print('FAIL')


#### Hard sudoku

In [ ]:
sudoku_hard = [
          [None, 7   , None, None, None, 8   , None, 6   , 3   ],
          [9   , None, None, None, 5   , None, 2   , None, None],
          [None, None, 1   , None, 2   , None, 7   , None, None],
          [None, None, None, 4   , None, None, None, None, None],
          [8   , None, 7   , None, None, 6   , None, None, None],
          [6   , 1   , None, None, None, None, None, None, None],
          [None, None, None, 1   , None, 3   , 5   , 9   , None],
          [7   , None, None, None, None, 2   , 8   , None, None],
          [None, None, 5   , None, None, None, None, None, 6   ]
] # HARD ONE

print('Solving a hard one...')
solved_hard = solve_sudoku(sudoku_hard)
if solved_hard['status']=='OK':
  solved_sudoku = solved_hard['sudoku']
  print(f"It is valid! Recursion depth required: {solved_hard['recursion_level']}" if is_valid_sudoku(solved_sudoku) else 'It is not valid...')
  print_sudoku(solved_sudoku)
else:
  print('FAIL')


#### Extreme sudoku

In [ ]:
sudoku_extreme = [
          [None, None, 6   , 7   , None, None, None, None, None],
          [None, 1   , None, None, 5   , None, None, 8   , 6   ],
          [3   , None, None, None, None, 6   , None, None, None],
          [6   , None, None, None, 1   , None, None, None, 2   ],
          [1   , 5   , 9   , None, None, 4   , None, None, None],
          [None, None, None, None, None, None, 8   , None, None],
          [None, None, None, 8   , None, None, None, None, 7   ],
          [None, None, None, None, None, None, 3   , None, None],
          [None, 2   , 1   , 6   , None, None, 9   , None, None]
] # EXTREME

print('Solving an extreme sudoku...')
solved_extreme = solve_sudoku(sudoku_extreme)
if solved_extreme['status']=='OK':
  solved_sudoku = solved_hard['sudoku']
  print(f"It is valid! Recursion depth required: {solved_extreme['recursion_level']}" if is_valid_sudoku(solved_sudoku) else 'It is not valid...')
  print_sudoku(solved_sudoku)
else:
  print('FAIL')

#### Blank sudoku matrix

In [12]:
empty_sudoku = get_empty_sudoku()
print('Creating one from scratch...')
solved_from_scratch = solve_sudoku(empty_sudoku)
if solved_from_scratch['status']=='OK':
  solved_sudoku = solved_from_scratch['sudoku']
  print(f"It is valid! Recursion depth required: {solved_from_scratch['recursion_level']}" if is_valid_sudoku(solved_sudoku) else 'It is not valid...')
  print_sudoku(solved_sudoku)
else:
  print('FAIL')

Creating one from scratch...
It is valid! Recursion depth required: 50

  ┌─────────┬─────────┬─────────┐
  │ 3  7  5 │ 8  9  2 │ 1  6  4 │
  │ 1  4  2 │ 5  7  6 │ 8  9  3 │
  │ 6  9  8 │ 3  4  1 │ 2  5  7 │
  ├─────────┼─────────┼─────────┤
  │ 8  5  3 │ 1  2  7 │ 6  4  9 │
  │ 2  1  4 │ 9  6  5 │ 3  7  8 │
  │ 7  6  9 │ 4  8  3 │ 5  2  1 │
  ├─────────┼─────────┼─────────┤
  │ 9  8  1 │ 2  5  4 │ 7  3  6 │
  │ 5  3  6 │ 7  1  9 │ 4  8  2 │
  │ 4  2  7 │ 6  3  8 │ 9  1  5 │
  └─────────┴─────────┴─────────┘
  
